<a href="https://colab.research.google.com/github/lauraluebbert/test_gget_alphafold/blob/main/test_gget_alphafold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [gget alphafold](https://github.com/pachterlab/gget) test

___

Install and import gget:

In [ ]:
!pip install -U -q gget
import gget

Use Miniconda to install OpenMM:  

In [ ]:
# For pretty plots
%config InlineBackend.figure_format='retina'

# Install Miniconda
import sys
!rm -rf /opt/conda
!wget -q -P /tmp \
  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh \
    && bash /tmp/Miniconda3-latest-Linux-x86_64.sh -b -p /opt/conda \
    && rm /tmp/Miniconda3-latest-Linux-x86_64.sh

# Install openmm v7.5.1
PATH=%env PATH
%env PATH=/opt/conda/bin:{PATH}
!conda install -qy conda==4.13.0 \
    && conda install -qy -c conda-forge \
      python=3.9 \
      openmm=7.5.1

# Add to path
sys.path.append('/opt/conda/lib/python3.9/site-packages')

PREFIX=/opt/conda
Unpacking payload ...
                                                                                                     
Installing base environment...





Preparing transaction: - \ | / - \ done
Executing transaction: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /opt/conda
env: PATH=/opt/conda/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible 

Install third-part dependencies and download AlphaFold model parameters using `gget setup` (this might take a few minutes):

In [5]:
gget.setup("alphafold")

######################################################################## 100.0%


Predict the 3D structure of CASP14 target [T1024](https://predictioncenter.org/casp14/target.cgi?id=8&view=all):

In [ ]:
gget.alphafold(
    "MAAHKGAEHHHKAAEHHEQAAKHHHAAAEHHEKGEHEQAAHHADTAYAHHKHAEEHAAQAAKHDAEHHAPKPH", 
    relax=True, 
    out=None
    )

Jackhmmer search:  22%|██▏       | 43/199 [elapsed: 08:16 remaining: 28:29]